In [100]:
import pandas as pd
import numpy as np

In [101]:
#export_germany 파일 불러오기
export_gm=pd.read_table('Trade_Map_-_List_of_products_exported_by_Germany.txt', sep='\t')
export_gm.rename(columns={'Code':'HS'}, inplace=True)
export_gm.drop(export_gm.index[0], inplace=True)
export_gm['HS']=export_gm['HS'].astype('int64')

In [102]:
#HS17 코드 불러와서 export_germany 파일과 병합
hs_bec=pd.read_csv('hs_bec17.csv')
export_gm=pd.merge(export_gm, hs_bec, how='left')

In [103]:
#최근 10년간 거래가 없는 품목 제외시킴
no_amount_gm=export_gm[(export_gm['Exported value in 2012']==0)
                       & (export_gm['Exported value in 2013']==0) 
                       & (export_gm['Exported value in 2014']==0) 
                       & (export_gm['Exported value in 2015']==0) 
                       & (export_gm['Exported value in 2016']==0) 
                       & (export_gm['Exported value in 2017']==0)  
                       & (export_gm['Exported value in 2018']==0) 
                       & (export_gm['Exported value in 2019']==0) 
                       & (export_gm['Exported value in 2020']==0) 
                       & (export_gm['Exported value in 2021']==0)].index
export_gm.drop(no_amount_gm, inplace=True)

In [104]:
#HS 999999품목은 따로 묶기 위해 임의의 BEC 번호 999999를 만들어 줌
export_gm.loc[export_gm['HS'] == 999999, 'BEC17'] = 999999

In [105]:
#BEC17 결측치 확인-81개
export_gm[export_gm['BEC17'].isnull()]

,HS,Product label,Exported value in 2012,Exported value in 2013,Exported value in 2014,Exported value in 2015,Exported value in 2016,Exported value in 2017,Exported value in 2018,Exported value in 2019,Exported value in 2020,Exported value in 2021,Unnamed: 12,BEC17
5106,401163,"Pneumatic tyres, new, of rubber, having a """"he...",22858,19990,20258,17065,14559,0,0,0,0,0,NaN,NaN
5107,401193,"Pneumatic tyres, new, of rubber, of a kind use...",5669,15388,13237,11549,2934,0,0,0,0,0,NaN,NaN
5108,401199,"Pneumatic tyres, new, of rubber (excluding hav...",21858,23784,20009,23149,33167,0,0,0,0,0,NaN,NaN
5141,600531,Unbleached or bleached warp knit fabrics of sy...,82435,91371,88938,74123,69487,0,0,0,0,0,NaN,NaN
5148,30625,"Norway lobsters """"Nephrops norvegicus"""", even ...",785,3619,3720,3626,2803,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6131,690890,"Glazed ceramic flags and paving, hearth or wal...",231482,226837,230519,194262,201041,0,0,0,0,0,NaN,NaN
6134,401169,"Pneumatic tyres, new, of rubber, having a """"he...",13480,12821,11709,4718,3211,0,0,0,0,0,NaN,NaN
6135,401194,"Pneumatic tyres, new, of rubber, of a kind use...",9508,5523,4256,6389,8111,0,0,0,0,0,NaN,NaN
6168,940381,Furniture of bamboo or rattan (excluding seats...,4340,4785,4623,6792,13200,0,0,0,0,0,NaN,NaN


In [106]:
#BEC17 결측치 있는 행들만 모아 데이터 프레임을 만듬(아래에서 BEC12 파일과 병합을 위해)
BEC17_nan_gm=export_gm[export_gm['BEC17'].isnull()]

In [107]:
#BEC12 파일과 BEC17 결측치가 담긴 데이터프레임을 병합
hs_bec12=pd.read_csv('hs_bec12.csv')
BEC17_nan_gm=pd.merge(BEC17_nan_gm, hs_bec12, how='left')

In [108]:
#BEC12와 모두 매치 되어 결측치 없음을 확인
BEC17_nan_gm.isnull().sum()

HS                         0
Product label              0
Exported value in 2012     0
Exported value in 2013     0
Exported value in 2014     0
Exported value in 2015     0
Exported value in 2016     0
Exported value in 2017     0
Exported value in 2018     0
Exported value in 2019     0
Exported value in 2020     0
Exported value in 2021     0
Unnamed: 12               81
BEC17                     81
BEC12                      0
dtype: int64

In [109]:
#BEC17로 채워진 행들(export_gm)과 BEC12(BEC17_nan_gm)로 채워진 행들을 합치기
export_gm_complete=pd.merge(export_gm, BEC17_nan_gm, how='left')

In [110]:
#컬럼들을 보기 좋게 배열하기 위해 컬럼명 확인
export_gm_complete.columns

Index(['HS', 'Product label', 'Exported value in 2012',
       'Exported value in 2013', 'Exported value in 2014',
       'Exported value in 2015', 'Exported value in 2016',
       'Exported value in 2017', 'Exported value in 2018',
       'Exported value in 2019', 'Exported value in 2020',
       'Exported value in 2021', 'Unnamed: 12', 'BEC17', 'BEC12'],
      dtype='object')

In [111]:
export_gm_complete=export_gm_complete[['HS', 'BEC17', 'BEC12', 'Product label', 'Exported value in 2012',
       'Exported value in 2013', 'Exported value in 2014',
       'Exported value in 2015', 'Exported value in 2016',
       'Exported value in 2017', 'Exported value in 2018',
       'Exported value in 2019', 'Exported value in 2020',
       'Exported value in 2021', 'Unnamed: 12']]


In [112]:
export_gm_complete

,HS,BEC17,BEC12,Product label,Exported value in 2012,Exported value in 2013,Exported value in 2014,Exported value in 2015,Exported value in 2016,Exported value in 2017,Exported value in 2018,Exported value in 2019,Exported value in 2020,Exported value in 2021,Unnamed: 12
0,999999,999999.0,NaN,Commodities not elsewhere specified,59373193,63816786,70174563,64670995,63831190,58021514,60664181,57559330,46910555,63274483,NaN
1,300490,711220.0,NaN,Medicaments consisting of mixed or unmixed pro...,36938587,38134155,39612106,35970611,37220220,42393474,45087259,45729836,49138138,52783884,NaN
2,870323,513202.0,NaN,Motor cars and other motor vehicles principall...,49549329,49490603,55114290,50015861,57000805,61580286,63825427,59214131,41983618,39686075,NaN
3,300215,711220.0,NaN,"Immunological products, put up in measured dos...",0,0,0,0,0,17544825,21685506,20062331,22519737,26220126,NaN
4,870340,513202.0,NaN,Motor cars and other motor vehicles principall...,0,0,0,0,0,115924,1589419,3733425,8230172,20431992,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5305,940381,NaN,313202.0,Furniture of bamboo or rattan (excluding seats...,4340,4785,4623,6792,13200,0,0,0,0,0,NaN
5306,844590,412010.0,NaN,Machines for producing textile yarns and machi...,16612,27452,0,0,0,0,0,0,0,0,NaN
5307,780419,311210.0,NaN,"Lead plates; lead sheets, strip and foil, of a...",52456,58428,57354,54093,61484,70634,69113,0,0,0,NaN
5308,852851,NaN,613202.0,Monitors of a kind solely or principally used ...,375039,404535,421819,398679,413527,0,0,0,0,0,NaN


In [113]:
#BEC17 컬럼과 BEC12 컬럼을 합쳐주기 위해 결측치를 0으로 처리
export_gm_complete['BEC17']=export_gm_complete['BEC17'].fillna(0)
export_gm_complete['BEC12']=export_gm_complete['BEC12'].fillna(0)
#BEC17 컬럼과 BEC12 컬럼의 값을 더해서 BEC 컬럼을 만들고 기존의 BEC17 컬럼과 BEC12 컬럼은 제외
export_gm_complete['BEC']=export_gm_complete['BEC17']+export_gm_complete['BEC12']
export_gm_complete=export_gm_complete.drop(columns=['BEC17','BEC12'])

In [114]:
export_gm_complete=export_gm_complete[['HS', 'BEC','Product label', 'Exported value in 2012',
       'Exported value in 2013', 'Exported value in 2014',
       'Exported value in 2015', 'Exported value in 2016',
       'Exported value in 2017', 'Exported value in 2018',
       'Exported value in 2019', 'Exported value in 2020',
       'Exported value in 2021', 'Unnamed: 12']]

In [115]:
export_gm_complete

,HS,BEC,Product label,Exported value in 2012,Exported value in 2013,Exported value in 2014,Exported value in 2015,Exported value in 2016,Exported value in 2017,Exported value in 2018,Exported value in 2019,Exported value in 2020,Exported value in 2021,Unnamed: 12
0,999999,999999.0,Commodities not elsewhere specified,59373193,63816786,70174563,64670995,63831190,58021514,60664181,57559330,46910555,63274483,NaN
1,300490,711220.0,Medicaments consisting of mixed or unmixed pro...,36938587,38134155,39612106,35970611,37220220,42393474,45087259,45729836,49138138,52783884,NaN
2,870323,513202.0,Motor cars and other motor vehicles principall...,49549329,49490603,55114290,50015861,57000805,61580286,63825427,59214131,41983618,39686075,NaN
3,300215,711220.0,"Immunological products, put up in measured dos...",0,0,0,0,0,17544825,21685506,20062331,22519737,26220126,NaN
4,870340,513202.0,Motor cars and other motor vehicles principall...,0,0,0,0,0,115924,1589419,3733425,8230172,20431992,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5305,940381,313202.0,Furniture of bamboo or rattan (excluding seats...,4340,4785,4623,6792,13200,0,0,0,0,0,NaN
5306,844590,412010.0,Machines for producing textile yarns and machi...,16612,27452,0,0,0,0,0,0,0,0,NaN
5307,780419,311210.0,"Lead plates; lead sheets, strip and foil, of a...",52456,58428,57354,54093,61484,70634,69113,0,0,0,NaN
5308,852851,613202.0,Monitors of a kind solely or principally used ...,375039,404535,421819,398679,413527,0,0,0,0,0,NaN


In [116]:
export_gm_complete.to_csv("export_gm.csv", index = False)